### **Zadanie: Refaktoryzacja kodu pod kątem SOLID**

Masz kod, który działa, ale łamie kilka zasad SOLID. Twoim zadaniem jest poprawienie go tak, aby przestrzegał tych zasad. Kod jest prosty, ale pełen potencjalnych problemów związanych z rozszerzalnością i utrzymaniem.

---

#### **Zastany kod:**
```python
class Order:
    def __init__(self, items, total_price):
        self.items = items
        self.total_price = total_price

    def calculate_discount(self, customer_type):
        if customer_type == "regular":
            return self.total_price * 0.9
        elif customer_type == "vip":
            return self.total_price * 0.8
        else:
            return self.total_price

    def generate_invoice(self):
        invoice = f"Invoice:\nItems: {self.items}\nTotal: {self.total_price}"
        with open("invoice.txt", "w") as file:
            file.write(invoice)
        return invoice

    def send_email(self, email):
        invoice = self.generate_invoice()
        print(f"Sending email to {email} with the following invoice:\n{invoice}")
```

---

#### **Twoje zadanie:**

1. **Zidentyfikuj problemy:**
   - Które zasady SOLID są łamane?
   - Dlaczego kod jest trudny do rozszerzenia i utrzymania?

2. **Zrefaktoryzuj kod:**
   - Zastosuj odpowiednie zasady SOLID.
   - Upewnij się, że kod jest elastyczny, łatwy w testowaniu i zgodny z dobrymi praktykami.

---

#### **Wynik oczekiwany:**

1. Kod powinien być podzielony na mniejsze, bardziej odpowiedzialne klasy.
2. Logika rabatów, generowania faktur i wysyłania e-maili powinna być oddzielona.
3. Nowe typy rabatów (np. „student”) powinny być łatwe do dodania bez modyfikowania istniejącego kodu.



### **Rozwiązanie: Refaktoryzacja kodu**

Oto poprawiona wersja kodu, która spełnia zasady SOLID:

---

#### **Zidentyfikowane problemy w zastanym kodzie:**
1. **Single Responsibility Principle (SRP)**:
   - Klasa `Order` ma za dużo odpowiedzialności: oblicza rabaty, generuje fakturę, zapisuje pliki i wysyła e-maile.
2. **Open/Closed Principle (OCP)**:
   - Dodanie nowego typu rabatu wymaga modyfikacji metody `calculate_discount`.
3. **Interface Segregation Principle (ISP)**:
   - Brak jasnego podziału na interfejsy dla różnych funkcji, np. rabatów, generowania faktur i wysyłania e-maili.
4. **Dependency Inversion Principle (DIP)**:
   - Klasa `Order` zależy od szczegółowej implementacji operacji na plikach i wysyłania e-maili.

---

#### **Refaktoryzowany kod:**

**1. Klasa reprezentująca zamówienie (SRP):**
```python
class Order:
    def __init__(self, items, total_price):
        self.items = items
        self.total_price = total_price
```

---

**2. Interfejs i implementacje rabatów (OCP):**
```python
from abc import ABC, abstractmethod

class Discount(ABC):
    @abstractmethod
    def apply_discount(self, total_price):
        pass

class RegularCustomerDiscount(Discount):
    def apply_discount(self, total_price):
        return total_price * 0.9

class VIPCustomerDiscount(Discount):
    def apply_discount(self, total_price):
        return total_price * 0.8

class NoDiscount(Discount):
    def apply_discount(self, total_price):
        return total_price
```

---

**3. Klasa odpowiedzialna za generowanie faktur (SRP):**
```python
class InvoiceGenerator:
    def generate_invoice(self, order):
        return f"Invoice:\nItems: {order.items}\nTotal: {order.total_price}"

    def save_to_file(self, invoice, filename="invoice.txt"):
        with open(filename, "w") as file:
            file.write(invoice)
```

---

**4. Klasa do wysyłania e-maili (SRP, DIP):**
```python
class EmailSender:
    def send_email(self, email, content):
        print(f"Sending email to {email} with the following content:\n{content}")
```

---

**5. Koordynacja w klasie wysokopoziomowej (DIP):**
```python
class OrderProcessor:
    def __init__(self, discount: Discount, invoice_generator: InvoiceGenerator, email_sender: EmailSender):
        self.discount = discount
        self.invoice_generator = invoice_generator
        self.email_sender = email_sender

    def process_order(self, order, email):
        # Oblicz rabat
        order.total_price = self.discount.apply_discount(order.total_price)

        # Generuj fakturę
        invoice = self.invoice_generator.generate_invoice(order)
        self.invoice_generator.save_to_file(invoice)

        # Wyślij fakturę e-mailem
        self.email_sender.send_email(email, invoice)
```

---

#### **Użycie zrefaktoryzowanego kodu:**
```python
# Stwórz zamówienie
order = Order(items=["Item1", "Item2"], total_price=100)

# Wybierz typ rabatu
discount = VIPCustomerDiscount()

# Stwórz generator faktur i wysyłanie e-maili
invoice_generator = InvoiceGenerator()
email_sender = EmailSender()

# Przetwórz zamówienie
processor = OrderProcessor(discount, invoice_generator, email_sender)
processor.process_order(order, email="customer@example.com")
```

---

### **Jak spełniono zasady SOLID:**

1. **SRP**: 
   - Klasa `Order` przechowuje tylko dane zamówienia.
   - Klasa `InvoiceGenerator` generuje faktury.
   - Klasa `EmailSender` obsługuje wysyłanie e-maili.
   
2. **OCP**: 
   - Dodanie nowego rabatu (np. `StudentDiscount`) wymaga tylko dodania nowej klasy, bez modyfikacji istniejących.

3. **LSP**:
   - Wszystkie klasy rabatów (`Discount`) mogą być użyte zamiennie.

4. **ISP**:
   - Klasy są małe i odpowiedzialne za jedno zadanie, więc interfejsy nie wymuszają implementacji zbędnych metod.

5. **DIP**:
   - `OrderProcessor` zależy od abstrakcji (`Discount`, `InvoiceGenerator`, `EmailSender`), a nie od szczegółowych implementacji.

---

### **Korzyści:**
- Łatwość dodawania nowych funkcji (np. nowych typów rabatów).
- Czytelność i możliwość testowania poszczególnych komponentów niezależnie.
- Uniknięcie efektu domina przy zmianach w kodzie.